In [1]:
import pandas as pd
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

### Lectura de polizas.feather

In [2]:
df_polizas = pd.read_feather('../../data/interim/polizas.feather')

In [3]:
df_polizas.nunique()

CIF_ID            899754
MCA_VIGENCIA           1
NUM_SECU_POL     9700101
COD_INICIADOR        749
FECHA_VIG_POL        579
dtype: int64

In [4]:
df_polizas.dtypes

CIF_ID           float64
MCA_VIGENCIA      object
NUM_SECU_POL       int64
COD_INICIADOR    float64
FECHA_VIG_POL     object
dtype: object

In [5]:
df_polizas['FECHA_VIG_POL'] = df_polizas['FECHA_VIG_POL'].map({date: pd.to_datetime(date, format="%d/%m/%Y") for date in df_polizas['FECHA_VIG_POL'].unique()})

In [6]:
data = {col: [df_polizas[col].min(),df_polizas[col].max()] for col in ['FECHA_VIG_POL']}
pd.DataFrame(data, index =['MIN', 'MAX'])

,FECHA_VIG_POL
MIN,2018-05-01
MAX,2019-11-30


*Calculamos mes y año de vigencia*

In [7]:
df_polizas['mes_anio_vig'] = df_polizas['FECHA_VIG_POL'].dt.strftime('%Y-%m')

In [8]:
sorted(df_polizas['mes_anio_vig'].unique())

['2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11']

*Filtramos las columnas para hacer el pivot por CIF_ID*

In [9]:
to_pivot = df_polizas[["CIF_ID","NUM_SECU_POL","MCA_VIGENCIA","mes_anio_vig"]].drop_duplicates() 

In [10]:
df_polizas_pivoted = to_pivot.pivot_table(index='CIF_ID',columns=['mes_anio_vig'], values=['MCA_VIGENCIA'],aggfunc='count',fill_value=0)

In [11]:
df_polizas_pivoted.head(5)

MCA_VIGENCIA                                                  \
mes_anio_vig      2018-05 2018-06 2018-07 2018-08 2018-09 2018-10 2018-11   
CIF_ID                                                                      
571.0                   0       0       0       0       0       0       1   
573.0                   0       0       0       0       0       0       1   
1009.0                  0       0       0       0       0       0       1   
1075.0                  0       0       0       0       0       0       0   
1103.0                  0       0       0       0       0       0       1   

                                                                              \
mes_anio_vig 2018-12 2019-01 2019-02 2019-03 2019-04 2019-05 2019-06 2019-07   
CIF_ID                                                                         
571.0              1       1       1       1       1       1       1       1   
573.0              1       1       1       1       1       1       1       1   
1009.0             1       1       1       1       1       1       1       1   
1075.0             0       0       0       0       0       0       0       0   
1103.0             1       1       1       1       1       1       1       1   

                                              
mes_anio_vig 2019-08 2019-09 2019-10 2019-11  
CIF_ID                                        
571.0              1       1       1       1  
573.0              1       1       1       1  
1009.0             1       1       1       1  
1075.0             1       1       1       1  
1103.0             1       1       1       1

In [12]:
df_polizas_pivoted = df_polizas_pivoted.iloc[:,-18:]

In [13]:
df_polizas_pivoted = df_polizas_pivoted.astype(str)

In [14]:
df_polizas_pivoted["history"] = df_polizas_pivoted.apply(" ".join, axis=1)

In [15]:
def simplify(x):
    return "".join(["1" if int(n)>0 else "0" for n in x.split(" ")])

df_polizas_pivoted["boolean_history"] = df_polizas_pivoted["history"].apply(simplify).values

In [16]:
df_polizas_pivoted.head(5)

MCA_VIGENCIA                                                  \
mes_anio_vig      2018-06 2018-07 2018-08 2018-09 2018-10 2018-11 2018-12   
CIF_ID                                                                      
571.0                   0       0       0       0       0       1       1   
573.0                   0       0       0       0       0       1       1   
1009.0                  0       0       0       0       0       1       1   
1075.0                  0       0       0       0       0       0       0   
1103.0                  0       0       0       0       0       1       1   

                                                                              \
mes_anio_vig 2019-01 2019-02 2019-03 2019-04 2019-05 2019-06 2019-07 2019-08   
CIF_ID                                                                         
571.0              1       1       1       1       1       1       1       1   
573.0              1       1       1       1       1       1       1       1   
1009.0             1       1       1       1       1       1       1       1   
1075.0             0       0       0       0       0       0       0       1   
1103.0             1       1       1       1       1       1       1       1   

                                                                  history  \
mes_anio_vig 2019-09 2019-10 2019-11                                        
CIF_ID                                                                      
571.0              1       1       1  0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1   
573.0              1       1       1  0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1   
1009.0             1       1       1  0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1   
1075.0             1       1       1  0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1   
1103.0             1       1       1  0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1   

                 boolean_history  
mes_anio_vig                      
CIF_ID                            
571.0         000001111111111111  
573.0         000001111111111111  
1009.0        000001111111111111  
1075.0        000000000000001111  
1103.0        000001111111111111

*Filtramos aquellos registros que nunca tienen polizas vigentes en el periodo analizado*

In [17]:
df_polizas_pivoted[df_polizas_pivoted["history"] == "0 0 0 0 0 0 0 0 0 0 0 0 0"].head()

Empty DataFrame
Columns: [(MCA_VIGENCIA, 2018-06), (MCA_VIGENCIA, 2018-07), (MCA_VIGENCIA, 2018-08), (MCA_VIGENCIA, 2018-09), (MCA_VIGENCIA, 2018-10), (MCA_VIGENCIA, 2018-11), (MCA_VIGENCIA, 2018-12), (MCA_VIGENCIA, 2019-01), (MCA_VIGENCIA, 2019-02), (MCA_VIGENCIA, 2019-03), (MCA_VIGENCIA, 2019-04), (MCA_VIGENCIA, 2019-05), (MCA_VIGENCIA, 2019-06), (MCA_VIGENCIA, 2019-07), (MCA_VIGENCIA, 2019-08), (MCA_VIGENCIA, 2019-09), (MCA_VIGENCIA, 2019-10), (MCA_VIGENCIA, 2019-11), (history, ), (boolean_history, )]
Index: []

In [18]:
df_polizas_pivoted = df_polizas_pivoted[df_polizas_pivoted["history"] != "0 0 0 0 0 0 0 0 0 0 0 0 0"]

*Ahora buscamos bajas y el periodo en el cual se detectan tres polizas seguidas sin vigencia*

In [19]:
df_polizas_pivoted["periodo_baja"] = -1

In [20]:
df_polizas_pivoted.loc[df_polizas_pivoted["periodo_baja"] == -1, "periodo_baja"] =\
df_polizas_pivoted.loc[df_polizas_pivoted["periodo_baja"] == -1, "boolean_history"].str.find(f'1000').values

In [21]:
df_polizas_pivoted.loc[df_polizas_pivoted["periodo_baja"] > -1,"periodo_baja"] =\
df_polizas_pivoted.loc[df_polizas_pivoted["periodo_baja"] > -1,"periodo_baja"] + 2 

**Just for checking:** observamos las bajas detectadas

In [22]:
df_polizas_pivoted[df_polizas_pivoted["periodo_baja"] > -1].head(15)

MCA_VIGENCIA                                                  \
mes_anio_vig      2018-06 2018-07 2018-08 2018-09 2018-10 2018-11 2018-12   
CIF_ID                                                                      
1429.0                  0       0       0       0       0       2       2   
5049.0                  0       0       0       0       0       1       1   
6964.0                  0       0       0       0       0       0       0   
8904.0                  0       0       0       0       0       1       1   
12648.0                 0       0       0       0       0       1       1   
13653.0                 0       0       0       0       0       1       1   
14283.0                 0       0       0       0       0       1       1   
15071.0                 0       0       0       0       0       1       1   
17383.0                 0       0       0       0       0       1       1   
18849.0                 0       0       0       0       0       1       1   
21519.0                 0       0       0       0       0       1       1   
24694.0                 0       0       0       0       0       1       1   
30533.0                 0       0       0       0       0       1       1   
33093.0                 0       0       0       0       0       1       1   
35308.0                 0       0       0       0       0       1       1   

                                                                              \
mes_anio_vig 2019-01 2019-02 2019-03 2019-04 2019-05 2019-06 2019-07 2019-08   
CIF_ID                                                                         
1429.0             2       0       0       0       0       0       0       0   
5049.0             1       1       1       0       0       0       0       0   
6964.0             0       0       0       0       1       1       0       0   
8904.0             1       1       1       0       0       0       1       1   
12648.0            0       0       0       0       0       0       0       0   
13653.0            2       1       1       1       1       1       0       0   
14283.0            1       1       1       1       1       1       0       0   
15071.0            0       0       0       0       0       0       0       0   
17383.0            1       1       1       0       0       0       0       0   
18849.0            1       1       1       0       0       0       0       0   
21519.0            1       1       1       0       0       0       0       0   
24694.0            1       0       0       0       0       0       0       0   
30533.0            1       0       0       0       0       0       0       0   
33093.0            1       1       1       1       1       1       1       0   
35308.0            1       1       1       1       1       1       1       0   

                                                                  history  \
mes_anio_vig 2019-09 2019-10 2019-11                                        
CIF_ID                                                                      
1429.0             0       0       0  0 0 0 0 0 2 2 2 0 0 0 0 0 0 0 0 0 0   
5049.0             0       0       0  0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0   
6964.0             0       0       0  0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0   
8904.0             1       1       1  0 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1   
12648.0            0       0       0  0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0   
13653.0            0       0       0  0 0 0 0 0 1 1 2 1 1 1 1 1 0 0 0 0 0   
14283.0            0       0       0  0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0   
15071.0            0       0       0  0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0   
17383.0            0       0       0  0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0   
18849.0            0       0       0  0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0   
21519.0            0       0       0  0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0   
24694.0            0       0       0  0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0   
30533.0            0       0  

In [23]:
df_bajas = pd.DataFrame(df_polizas_pivoted.index)

In [24]:
df_bajas = df_bajas.set_index('CIF_ID')

In [25]:
df_bajas["hist_polizas"] = df_polizas_pivoted["history"]
df_bajas["periodo_baja"] = df_polizas_pivoted["periodo_baja"]

In [26]:
df_bajas

,hist_polizas,periodo_baja
CIF_ID,,
571.0,0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1,-1
573.0,0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1,-1
1009.0,0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1,-1
1075.0,0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1,-1
1103.0,0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1,-1
1313.0,0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1,-1
1429.0,0 0 0 0 0 2 2 2 0 0 0 0 0 0 0 0 0 0,9
1482.0,0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1,-1
1559.0,0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1,-1


In [27]:
len(df_bajas)

899754

In [28]:
df_bajas.reset_index().to_feather('../../data/processed/periodos_baja_x_cif_id.feather')